In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%tensorflow_version 2.x
import tensorflow as tf
import torch
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
from drive.MyDrive.how_good_is_my_plot.fine_tune_for_plots import train_model, initialize_model
from drive.MyDrive.how_good_is_my_plot.regression_dataset import RegressionDataset

from __future__ import print_function
from __future__ import division

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import time
import os
import copy
import numpy as np


def train_and_test(data_dir, num_epochs, learning_rate, regularization, batch_size, phase):

    # Initialize the model for this run
    num_classes = 1
    input_size = 224
    model_ft = initialize_model(num_classes, use_pretrained=True)

    # Resizing and normalizing data
    data_transforms = transforms.Compose([
            transforms.Resize((input_size, input_size)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    print("Initializing Datasets and Dataloaders...")

    # Create training and validation datasets
    image_datasets = {x: RegressionDataset(data_dir, f"drive/MyDrive/how_good_is_my_plot/csv_splits/{phase}_{x}.csv", data_transforms) for x in ['train', 'val']}

    # Create training and validation dataloaders
    dataloaders_dict = {
        x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=2) for x in
        ['train', 'val']}

    # Detect if we have a GPU available
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Send the model to GPU
    model_ft = model_ft.to(device)

    params_to_update = model_ft.parameters()

    optimizer_ft = optim.Adam(params_to_update, lr=learning_rate, weight_decay=regularization)

    # Setup the loss fxn
    criterion = nn.MSELoss()

    # Train
    model, best_val_loss = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, device, num_epochs=num_epochs, regression=True)

    # Test
    test_dataset = RegressionDataset(data_dir, f"drive/MyDrive/how_good_is_my_plot/csv_splits/{phase}_test.csv", data_transforms)

    # Create test dataloader
    dataloader_test = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

    criterion = nn.MSELoss(reduction='none')

    test_loss = []
    for inputs, labels in dataloader_test:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)

        loss = criterion(outputs, labels)
        test_loss.append(loss.flatten().tolist())
    
    # test_loss /= len(dataloader_test.dataset)
    test_loss = [l for list in test_loss for l in list]
    
    return test_loss, best_val_loss

def training_parameters_choice(learning_rates, lambdas, batch_sizes):

    num_epochs = 30
    num_classes = 1
    data_dir = f"drive/MyDrive/how_good_is_my_plot/final_plots_forMT"

    best_parameters = None
    best_loss = 1e10

    for lr in learning_rates:
      for reg in lambdas:
        for bs in batch_sizes:
          current_loss = 0
          for split in range(1, 6):
            phase = f'split{split}'
            print(f'Currently on {phase} with parameters: {lr, reg, bs}')
            test, _ = train_and_test(data_dir, num_epochs, lr, reg, bs, phase)
            current_loss += np.mean(test)
          current_loss /= 5

          if current_loss < best_loss:
            best_parameters = [lr, reg, bs]

    return best_parameters


In [ ]:
learning_rates = [1e-4, 1e-5, 1e-6]
lambdas = [1e-3, 1e-5]
batch_sizes = [20, 50]

best_parameters = training_parameters_choice(learning_rates, lambdas, batch_sizes)
# best_parameters = [1e-05, 1e-05, 50]
print('BEST PARAMETERS:', best_parameters)

data_dir = f"drive/MyDrive/how_good_is_my_plot/final_plots_forMT"
test_loss, best_val_loss = train_and_test(data_dir, 30, *best_parameters, 'final')

print('Best validation loss: ', best_val_loss)
print('MSE on test data: ', np.mean(test_loss), '+-', np.std(test_loss)/np.sqrt(len(test_loss)))

Currently on split1 with parameters: (0.0001, 0.001, 20)
Initializing Datasets and Dataloaders...
Epoch 0/29
----------
train Loss: 172.3238
val Loss: 42.5847

Epoch 1/29
----------
train Loss: 149.6264
val Loss: 39.3108

Epoch 2/29
----------
train Loss: 126.0687
val Loss: 66.5159

Epoch 3/29
----------
train Loss: 106.4935
val Loss: 56.1858

Epoch 4/29
----------
train Loss: 70.4726
val Loss: 43.8947

Epoch 5/29
----------
train Loss: 59.0512
val Loss: 50.4508

Epoch 6/29
----------
train Loss: 51.0617
val Loss: 45.5927

Epoch 7/29
----------
train Loss: 47.5581
val Loss: 37.4330

Epoch 8/29
----------
train Loss: 38.2884
val Loss: 33.7895

Epoch 9/29
----------
train Loss: 34.8576
val Loss: 39.7444

Epoch 10/29
----------
train Loss: 30.3478
val Loss: 63.6899

Epoch 11/29
----------
train Loss: 28.6992
val Loss: 39.3737

Epoch 12/29
----------
train Loss: 20.6549
val Loss: 26.8478

Epoch 13/29
----------
train Loss: 27.1849
val Loss: 30.8099

Epoch 14/29
----------
train Loss: 19.44